In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from Modules.validaciones import *
import calendar as cl
import calendar

# Limpieza de datos

In [7]:
#leo los archivos
Detalle_op= pd.read_csv('../datos/202106/base_op_pami.csv')
sql = pd.read_csv('../datos/202106/base_sql.csv')
codigos_txt = pd.read_csv('../datos/202106/CODIGOS_TXT.csv')
visitas = pd.read_csv('../datos/202106/visitas_pami.csv')
Arch_auxiliar = pd.read_csv('../datos/202106/2021_07_ARCH_Auxiliares_valores.csv')



In [3]:
#renombro la columna
Arch_auxiliar = Arch_auxiliar.rename(columns={'Ctrol Afiliado 1': 'BENEFICIO'})

In [4]:
#borro la columna exra
Arch_auxiliar = Arch_auxiliar.drop(['Unnamed: 34'], axis=1)

In [5]:
#convierto
#Arch_auxiliar['NRO. OP'] = Arch_auxiliar['NRO. OP'].astype(str)
Arch_auxiliar['Parentesco'] = Arch_auxiliar['Parentesco'].astype(str)
sql['numdoc'] = sql['numdoc'].astype(str)

#cnvierto a fechas
Arch_auxiliar['Activada'] = pd.to_datetime(Arch_auxiliar['Activada'], dayfirst=True, format="%d/%m/%Y")
Arch_auxiliar['F. SOLICITUD'] = pd.to_datetime(Arch_auxiliar['F. SOLICITUD'], dayfirst=True, format="%d/%m/%Y")
Arch_auxiliar['Activada'] = pd.to_datetime(Arch_auxiliar['Activada'],dayfirst=True, format="%d/%m/%Y")
Arch_auxiliar['Generada/Aprobado'] = pd.to_datetime(Arch_auxiliar['Generada/Aprobado'])
Arch_auxiliar['Fecha de Inicio'] = pd.to_datetime(Arch_auxiliar['Fecha de Inicio'])
Arch_auxiliar['Fecha de fin'] = pd.to_datetime(Arch_auxiliar['Fecha de fin'])


Funciones

In [6]:
#concateno benef/paren
def concat_b_p(b):
    xs = b.split()
    concat = xs[0] + xs[2]
    return concat

In [7]:
Arch_auxiliar['NRO. BENEFICIO/GP'] = Arch_auxiliar['NRO. BENEFICIO/GP'].apply(concat_b_p)

In [9]:
#Agrego ceros 
Arch_auxiliar['NRO. BENEFICIO/GP'] = add_zeros_to_left(Arch_auxiliar, 'NRO. BENEFICIO/GP', 14)
Arch_auxiliar['BENEFICIO'] = add_zeros_to_left(Arch_auxiliar, 'BENEFICIO', 12)
Arch_auxiliar['Parentesco'] = add_zeros_to_left(Arch_auxiliar, 'Parentesco', 2)

In [12]:
#convertir ciertas fechas en null
fechas = ['Fecha de Inicio', 'Fecha de fin', 'Fecha inicio INT 1', 'Fecha FIN INT 1', 'Fecha inicio INT 2', 'Fecha FIN INT 2','Fecha inicio INT 3', 'Fecha FIN INT 3','OBITO']
sin_fechas = ['S/AGENDA', 'S/INTERNACION', 'S/OBITO', 'S/D', 'OBITO', 'SIN ALTA']
Arch_auxiliar[fechas] = Arch_auxiliar[fechas].applymap(lambda x: None if x in sin_fechas else x)

In [13]:
Arch_auxiliar[fechas] = Arch_auxiliar[fechas].apply(pd.to_datetime)

In [14]:
Arch_auxiliar['NRO. BENEFICIO/GP'] = Arch_auxiliar['NRO. BENEFICIO/GP'].astype(int)

In [15]:
Arch_auxiliar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 34 columns):
 #   Column                                                 Non-Null Count  Dtype         
---  ------                                                 --------------  -----         
 0   UGL                                                    242 non-null    object        
 1   AGENCIA                                                236 non-null    object        
 2   NRO. OP                                                242 non-null    int64         
 3   MOTIVO DE EMISION                                      242 non-null    object        
 4   PRESTADOR DE RECHAZO                                   0 non-null      float64       
 5   F. SOLICITUD                                           242 non-null    datetime64[ns]
 6   NRO. BENEFICIO/GP                                      242 non-null    int64         
 7   APELLIDO Y NOMBRE                                      242 non-null    

# Cabecera op

In [16]:
#leo las columnas
Arch_auxiliar.columns

Index(['UGL', 'AGENCIA', 'NRO. OP', 'MOTIVO DE EMISION',
       'PRESTADOR DE RECHAZO', 'F. SOLICITUD', 'NRO. BENEFICIO/GP',
       'APELLIDO Y NOMBRE', 'SOLICITANTE', 'PRESTADOR', 'ESTADO',
       'ESTADO DE PRACTICA: APROBADA', 'ESTADO DE PRACTICA: RECHAZADA',
       'ESTADO DE PRACTICA: SE SOLICITA INFORMACION ADICIONAL',
       'ESTADO DE PRACTICA: ASIGNAR TURNO', 'ESTADO DE PRACTICA: CONFORMAR',
       'ESTADO DE PRACTICA: NO CONFORMAR', 'PENDIENTE DE AUTORIZACION',
       'USUARIO EMISOR', 'BENEFICIO', 'Ctrol Afiliado 2', 'Parentesco',
       'DIAGNOTICO', 'Generada/Aprobado', 'Activada', 'Fecha de Inicio',
       'Fecha de fin', 'Fecha inicio INT 1', 'Fecha FIN INT 1',
       'Fecha inicio INT 2', 'Fecha FIN INT 2', 'Fecha inicio INT 3',
       'Fecha FIN INT 3', 'OBITO'],
      dtype='object')

In [17]:
#traigo las columnas que necesito para el archivo final 
Archivo_final = Arch_auxiliar[['APELLIDO Y NOMBRE','NRO. OP', 'NRO. BENEFICIO/GP','BENEFICIO', 'Parentesco','DIAGNOTICO','F. SOLICITUD', 'Activada','Fecha de Inicio', 'Fecha de fin', 'Fecha inicio INT 1', 'Fecha FIN INT 1', 'Fecha inicio INT 2', 'Fecha FIN INT 2', 'Fecha inicio INT 3', 'Fecha FIN INT 3', 'OBITO']]

In [18]:
#agrego el resto de laArchivo_final = s columnas
Archivo_final[['Periodo de alta', 'Tipo de documento', 'Nro Documento', 'Fecha Nacimiento','Sexo','Estado', 'Obs_Estado']] = " "

<ipython-input-18-aa339854f60a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Archivo_final[['Periodo de alta', 'Tipo de documento', 'Nro Documento', 'Fecha Nacimiento','Sexo','Estado', 'Obs_Estado']] = " "
/home/sabri/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [19]:
Archivo_final.columns


Index(['APELLIDO Y NOMBRE', 'NRO. OP', 'NRO. BENEFICIO/GP', 'BENEFICIO',
       'Parentesco', 'DIAGNOTICO', 'F. SOLICITUD', 'Activada',
       'Fecha de Inicio', 'Fecha de fin', 'Fecha inicio INT 1',
       'Fecha FIN INT 1', 'Fecha inicio INT 2', 'Fecha FIN INT 2',
       'Fecha inicio INT 3', 'Fecha FIN INT 3', 'OBITO', 'Periodo de alta',
       'Tipo de documento', 'Nro Documento', 'Fecha Nacimiento', 'Sexo',
       'Estado', 'Obs_Estado'],
      dtype='object')

In [20]:
#Reordeno las columnas
Cabecera_op  = Archivo_final[['Periodo de alta','APELLIDO Y NOMBRE', 'Tipo de documento', 'Nro Documento', 'Fecha Nacimiento', 'Sexo', 'NRO. OP', 'NRO. BENEFICIO/GP', 'BENEFICIO',
       'Parentesco', 'DIAGNOTICO', 'F. SOLICITUD', 'Activada','Fecha de Inicio', 'Fecha de fin', 'Fecha inicio INT 1','Fecha FIN INT 1', 'Fecha inicio INT 2', 'Fecha FIN INT 2',
       'Fecha inicio INT 3', 'Fecha FIN INT 3', 'OBITO','Estado', 'Obs_Estado']].copy()

In [21]:
Cabecera_op.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Periodo de alta     242 non-null    object        
 1   APELLIDO Y NOMBRE   242 non-null    object        
 2   Tipo de documento   242 non-null    object        
 3   Nro Documento       242 non-null    object        
 4   Fecha Nacimiento    242 non-null    object        
 5   Sexo                242 non-null    object        
 6   NRO. OP             242 non-null    int64         
 7   NRO. BENEFICIO/GP   242 non-null    int64         
 8   BENEFICIO           242 non-null    object        
 9   Parentesco          242 non-null    object        
 10  DIAGNOTICO          242 non-null    object        
 11  F. SOLICITUD        242 non-null    datetime64[ns]
 12  Activada            242 non-null    datetime64[ns]
 13  Fecha de Inicio     242 non-null    datetime64[ns]

Relleno las columnas de Apellido y nombre, tipo doc, doc, fecha nacimiento y sexo (Posteriormente vendran en el archivo en casa?)

In [22]:
def relleno(op):
    df = sql[sql['nro_autorizacion'] == op]
    tipo= []
    doc = []
    fecha_nacimiento = []
    sexo = []
    
    tipo.append(df['tipodoc'].min())
    doc.append(df['numdoc'].min())
    fecha_nacimiento.append(df['fecha_nacimiento'].min())
    sexo.append(df['sexo'].min())
    
    return tipo, doc, fecha_nacimiento, sexo
    
      

In [23]:
listas = Cabecera_op['NRO. OP'].apply(relleno)

In [24]:
listas= list(listas)

In [25]:
#listas

In [26]:
lista1 = []
lista2 = []
lista3 = []
lista4 = []

for i in listas:
    lista1.append(i[0][0])
    lista2.append(i[1][0])
    lista3.append(i[2][0])
    lista4.append(i[3][0])


Cabecera_op['Tipo de documento'] = pd.Series(lista1)
Cabecera_op['Nro Documento'] = pd.Series(lista2) 
Cabecera_op['Fecha Nacimiento'] = pd.Series(lista3)
Cabecera_op['Sexo'] = pd.Series(lista4)


In [27]:
#remplazo el sexo mal copiado
Cabecera_op['Sexo'] = Cabecera_op['Sexo'].replace('METRO', 'M')

In [28]:
Cabecera_op

,Periodo de alta,APELLIDO Y NOMBRE,Tipo de documento,Nro Documento,Fecha Nacimiento,Sexo,NRO. OP,NRO. BENEFICIO/GP,BENEFICIO,Parentesco,...,Fecha de fin,Fecha inicio INT 1,Fecha FIN INT 1,Fecha inicio INT 2,Fecha FIN INT 2,Fecha inicio INT 3,Fecha FIN INT 3,OBITO,Estado,Obs_Estado
0,,GELADA ESTER DEL CARMEN,LC,4593735,6/11/1944,F,9919050149,15021309770600,150213097706,00,...,2021-09-29,NaT,NaT,NaT,NaT,NaT,NaT,NaT,,
1,,SAA MERCEDES JUANA,DI,11974849,29/12/1955,F,9919032395,15511500381400,155115003814,00,...,2021-05-08,NaT,NaT,NaT,NaT,NaT,NaT,2021-05-08,,
2,,GOMEZ MARIA BEATRIZ,DI,4123576,5/3/1941,F,9919030927,15028009310300,150280093103,00,...,2021-08-08,NaT,NaT,NaT,NaT,NaT,NaT,2021-08-08,,
3,,MOLINA RAUL GERONIMO,DI,6689150,29/6/1941,M,9919025284,15030708170300,150307081703,00,...,2021-10-26,NaT,NaT,NaT,NaT,NaT,NaT,NaT,,
4,,CABRERA ADEMAR OSVALDO,DI,11089336,28/2/1954,M,9919021137,14092243040900,140922430409,00,...,2021-06-08,NaT,NaT,NaT,NaT,NaT,NaT,2021-06-08,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,,PUERTA GUILLERMO DAVID,DI,6478232,8/6/1932,F,9918141457,13053723080100,130537230801,00,...,2021-05-07,NaT,NaT,NaT,NaT,NaT,NaT,NaT,,
238,,LUJAN MARGARITA,DI,3968018,8/6/1940,F,9918039154,15016867850000,150168678500,00,...,2021-06-07,NaT,NaT,NaT,NaT,NaT,NaT,NaT,,
239,,GARAY AMALIA,LC,2963908,9/1/1933,F,9918032269,9561437260300,095614372603,00,...,2021-09-07,NaT,NaT,NaT,NaT,NaT,NaT,NaT,,
240,,CORTEZ SUSANA CATALINA,DI,6029828,29/10/1939,F,9918009330,15045427390000,150454273900,00,...,2021-01-07,NaT,NaT,NaT,NaT,NaT,NaT,NaT,,


Relleno la columna Periodo de alta

In [29]:
fecha = datetime.strptime("07/01/21", "%m/%d/%y")

In [30]:
fecha

datetime.datetime(2021, 7, 1, 0, 0)

In [31]:
def periodo_alta(activada,fecha):
    if activada < fecha:
        return ''
    else:
        return fecha
        
      

In [32]:
Cabecera_op['Periodo de alta'] = Cabecera_op['Activada'].apply(periodo_alta, fecha = datetime.strptime("07/01/21", "%m/%d/%y"))

In [33]:
Cabecera_op.head()

,Periodo de alta,APELLIDO Y NOMBRE,Tipo de documento,Nro Documento,Fecha Nacimiento,Sexo,NRO. OP,NRO. BENEFICIO/GP,BENEFICIO,Parentesco,...,Fecha de fin,Fecha inicio INT 1,Fecha FIN INT 1,Fecha inicio INT 2,Fecha FIN INT 2,Fecha inicio INT 3,Fecha FIN INT 3,OBITO,Estado,Obs_Estado
0,2021-07-01,GELADA ESTER DEL CARMEN,LC,4593735,6/11/1944,F,9919050149,15021309770600,150213097706,00,...,2021-09-29,NaT,NaT,NaT,NaT,NaT,NaT,NaT,,
1,2021-07-01,SAA MERCEDES JUANA,DI,11974849,29/12/1955,F,9919032395,15511500381400,155115003814,00,...,2021-05-08,NaT,NaT,NaT,NaT,NaT,NaT,2021-05-08,,
2,2021-07-01,GOMEZ MARIA BEATRIZ,DI,4123576,5/3/1941,F,9919030927,15028009310300,150280093103,00,...,2021-08-08,NaT,NaT,NaT,NaT,NaT,NaT,2021-08-08,,
3,2021-07-01,MOLINA RAUL GERONIMO,DI,6689150,29/6/1941,M,9919025284,15030708170300,150307081703,00,...,2021-10-26,NaT,NaT,NaT,NaT,NaT,NaT,NaT,,
4,2021-07-01,CABRERA ADEMAR OSVALDO,DI,11089336,28/2/1954,M,9919021137,14092243040900,140922430409,00,...,2021-06-08,NaT,NaT,NaT,NaT,NaT,NaT,2021-06-08,,


In [34]:
#LO saco para corroborar
#Cabecera_op.to_csv('../datos/Cabecera_prueba.csv',header=True, index=False)

Funciones para calcular la fecha de inicio

In [36]:
#Dato del periodo de calculo
periodo = datetime.strptime("07/01/21", "%m/%d/%y")

In [37]:
#Filtro las op que necesito calcular la fecha de inicio 
op_nuevas = Cabecera_op[Cabecera_op['Periodo de alta'] == periodo]

In [38]:
op_nuevas

,Periodo de alta,APELLIDO Y NOMBRE,Tipo de documento,Nro Documento,Fecha Nacimiento,Sexo,NRO. OP,NRO. BENEFICIO/GP,BENEFICIO,Parentesco,...,Fecha de fin,Fecha inicio INT 1,Fecha FIN INT 1,Fecha inicio INT 2,Fecha FIN INT 2,Fecha inicio INT 3,Fecha FIN INT 3,OBITO,Estado,Obs_Estado
0,2021-07-01,GELADA ESTER DEL CARMEN,LC,4593735,6/11/1944,F,9919050149,15021309770600,150213097706,00,...,2021-09-29,NaT,NaT,NaT,NaT,NaT,NaT,NaT,,
1,2021-07-01,SAA MERCEDES JUANA,DI,11974849,29/12/1955,F,9919032395,15511500381400,155115003814,00,...,2021-05-08,NaT,NaT,NaT,NaT,NaT,NaT,2021-05-08,,
2,2021-07-01,GOMEZ MARIA BEATRIZ,DI,4123576,5/3/1941,F,9919030927,15028009310300,150280093103,00,...,2021-08-08,NaT,NaT,NaT,NaT,NaT,NaT,2021-08-08,,
3,2021-07-01,MOLINA RAUL GERONIMO,DI,6689150,29/6/1941,M,9919025284,15030708170300,150307081703,00,...,2021-10-26,NaT,NaT,NaT,NaT,NaT,NaT,NaT,,
4,2021-07-01,CABRERA ADEMAR OSVALDO,DI,11089336,28/2/1954,M,9919021137,14092243040900,140922430409,00,...,2021-06-08,NaT,NaT,NaT,NaT,NaT,NaT,2021-06-08,,
5,2021-07-01,ORDOÑEZ ZULEMA MARIA,DI,5587939,3/4/1947,F,9919010613,15041759610000,150417596100,00,...,2021-02-08,NaT,NaT,NaT,NaT,NaT,NaT,2021-02-08,,
6,2021-07-01,NUÑEZ FERMINA FILOMENA,DI,10764806,25/9/1953,F,9919005816,15060873770500,150608737705,00,...,2021-10-28,NaT,NaT,NaT,NaT,NaT,NaT,NaT,,
7,2021-07-01,BARDUS MARIA ANTONIA,DI,2453253,4/8/1937,F,9919003802,15027414820100,150274148201,00,...,2021-10-28,NaT,NaT,NaT,NaT,NaT,NaT,NaT,,
8,2021-07-01,MOLINA CARLOS HUGO,DI,7979869,23/10/1944,M,9918983260,15046289060300,150462890603,00,...,2021-08-23,NaT,NaT,NaT,NaT,NaT,NaT,NaT,,
9,2021-07-01,GONZALEZ MARIA ROSA,DI,5294769,7/10/1945,F,9918974530,15026149150200,150261491502,00,...,2021-10-26,NaT,NaT,NaT,NaT,NaT,NaT,NaT,,


In [39]:
def requiere_visita(op):
    filtro = Detalle_op[(Detalle_op['NRO. OP'] == op) & (Detalle_op['Visita'] == 'si')]
            
    if len(filtro) <= 0:
        return False
    else:
        return True
    

In [40]:
requiere_visita(9919032395)

True

In [41]:
#funcion que devuelve los prestadores para ese modulo
def Tipos_servicio_prestador_OP(OP,practica_op):
    df = Detalle_op[Detalle_op['NRO. OP'] == OP]
    df2 = codigos_txt[codigos_txt['CODIGO DE PRACTICA OP'] == practica_op]
    df3 = pd.merge(df,df2,left_on=['CODIGO DE PRACTICA OP'],right_on=['CODIGO DE PRACTICA OP'])
        
    return df3['NRO. BENEFICIO-GP'].min() , list(df3['Tipo servicio prestador'].dropna())

    

In [42]:
datos = Tipos_servicio_prestador_OP(9919032395, 214001)

In [43]:
datos

(15511500381400, ['Médica/o', 'Enfermera/o'])

In [39]:
def Primer_visita(Beneficio, periodo_alta, prestador):
    Fecha_inicio= ''
    
    if requiere_visita(int(f['NRO. OP'])): #si no requiere visita
        Fecha_inicio = f['Activada']
    else:                                  #si requiere visita
        activada = str(f['Activada'])
        b = visitas[(visitas['N° afiliación'] == Beneficio) & (visitas['Fecha comienzo'] >= activada)]
        if all(item in prestador for item in list(b['Tipo servicio prestador'])):
            Fecha_inicio = b['Fecha comienzo'].min()
        else:
            Fecha_inicio = ''
            
    return Fecha_inicio 
    

In [40]:
#Version 2 de primer visita sin usar la funcion tipo servicio prestador 
def Primer_visita(Beneficio, periodo_alta):
    Fecha_inicio= ''
    
    f = Cabecera_op[(Cabecera_op['NRO. BENEFICIO/GP'] == Beneficio) & (Cabecera_op['Periodo de alta'] == periodo_alta)]
      
    
    if requiere_visita(int(f['NRO. OP'])): #si no requiere visita
        Fecha_inicio = f['Activada']
    else:                                  #si requiere visita
        activada = str(f['Activada'])
        b = visitas[(visitas['N° afiliación'] == Beneficio) & (visitas['Fecha comienzo'] >= activada) & ( visitas['Tipo servicio prestador'] == 'Enfermera/o' or  visitas['Tipo servicio prestador'] == 'Médica/o')]
        if len(b)>=1:
            Fecha_inicio = b['Fecha comienzo'].min()
        else:
            Fecha_inicio = ''
            
    return Fecha_inicio 


In [41]:
Primer_visita(15511500381400, periodo)

1   2021-07-28
Name: Activada, dtype: datetime64[ns]

In [42]:
#Agregar columna primer visita

In [671]:
#Calculo el estado de los obitos
def obito(o):
    if o == 'S/OBITO':
        return ''
    elif o == 'S/D':
        return''
    else:
        return'VALIDA'


In [672]:
Cabecera_op['Estado'] = Cabecera_op['OBITO'].apply(obito)

In [673]:
Cabecera_op['Obs_Estado'] = Cabecera_op['Estado'].apply(lambda x: 'OBITO' if x== 'VALIDA' else '')